# CIFAR-10 Image Classification
This notebook implements a training and testing pipeline for an image classification task on the CIFAR-10 dataset. CIFAR-10 contains 60,000 32x32 RGB images distributed evenly across 10 image classes (6,000 images per class). The provided dataset splits consists of a train set with 50,000 images and a test set with 10,000 images. Here, the train set is further split into a train set with 45,000 images and a validation set with 5,000 images to allow for model evaluation throughout the training process. The model currently used is a barebones CNN architecture (model architecture will be periodically updated with progress).
## Milestones
- 10/23: init + setup basic training pipeline
- 10/24: achieve ~50% test accuracy with a barebones CNN

## Setup
Import essential libraries (PyTorch) and declare hyperparameters.

In [350]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm

BATCH_SIZE = 128
EPOCHS = 20
LEARNING_RATE = 1e-4
CHECKPOINT_FOLDER = 'checkpoint'
DEVICE = torch.device('cpu')

## Data
Load the CIFAR-10 dataset using torchvision's dataset utilities. Apply normalization based on computed mean and std of the training set (see the commented out cell below). Initialize the train, validation, and test dataloaders.

In [351]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

cifar_iter = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
train_iter, val_iter = torch.utils.data.random_split(cifar_iter, [45000, 5000])
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, shuffle=False)

test_iter = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
test_dataloader = DataLoader(test_iter, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [352]:
# print(cifar_iter.data.shape)
# data = torch.tensor(cifar_iter.data, dtype=torch.float32)
# mean = torch.mean(data, dim=(0, 1, 2)) / 255
# std = torch.std(data, dim=(0, 1, 2)) / 255
# print(mean, std)

## Classification Model
Implement a barebones CNN architecture with a sequential linear classification head. Uses relu activations for convolutional and linear layers.

In [353]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.linear = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
        )
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = torch.flatten(x, 1)
        x = self.linear(x)
        return x

# Train, Evaluate, Score
- train_epoch: implements the training loop, which inputs images into the model, computes the cross entropy loss between the model outputs and labels, and backpropagates
- evaluate: implements the validation loop, which evaluates the model performance with the cross entropy loss as the metric (temporarily)
- score: implements the testing loop, which computes the trained model's performance on the unseen test data

In [354]:
def train_epoch(train_dataloader: DataLoader, model: ImageClassifier, optimizer):
    model.train()
    losses = 0

    for images, labels in tqdm(train_dataloader):
        images.to(DEVICE)
        labels.to(DEVICE)

        logits = model(images)
        loss = F.cross_entropy(logits, labels)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        losses += loss.item()

    return losses / len(train_dataloader)

In [355]:
def evaluate(val_dataloader: DataLoader, model: ImageClassifier):
    model.eval()
    losses = 0

    for images, labels in tqdm(val_dataloader):
        images.to(DEVICE)
        labels.to(DEVICE)

        logits = model(images)
        loss = F.cross_entropy(logits, labels)

        losses += loss.item()

    return losses / len(val_dataloader)

In [356]:
def score(test_dataloader: DataLoader, model: ImageClassifier):
    model.eval()
    losses = 0
    acc = 0

    for images, labels in test_dataloader:
        images.to(DEVICE)
        labels.to(DEVICE)

        logits = model(images)
        loss = loss_fn(logits, labels)

        losses += loss.item()
        _, max = torch.max(logits, dim=-1)
        acc += torch.sum(max == labels).item()
        
    return losses / len(test_dataloader), acc / len(test_iter)

In [357]:
model = ImageClassifier()
model.to(DEVICE)

optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

for epoch in range(EPOCHS):
    train_loss = train_epoch(train_dataloader, model, optimizer)
    val_loss = evaluate(val_dataloader, model)
    print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
    }, CHECKPOINT_FOLDER + f'/cifar_epoch{epoch}.pt')

  0%|          | 0/352 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:00<00:00, 40.52it/s]


Epoch: 0, Train loss: 2.125, Val loss: 1.979


100%|██████████| 40/40 [00:00<00:00, 40.66it/s]


Epoch: 1, Train loss: 1.898, Val loss: 1.812


...

Epoch: 17, Train loss: 1.383, Val loss: 1.408


100%|██████████| 40/40 [00:00<00:00, 42.94it/s]


Epoch: 18, Train loss: 1.371, Val loss: 1.404


100%|██████████| 40/40 [00:00<00:00, 40.47it/s]

Epoch: 19, Train loss: 1.359, Val loss: 1.390


In [358]:
test_loss, test_acc = score(test_dataloader, model)
print(test_loss, test_acc)

1.356563856330099 0.5149
